# Challenge SRL Models - Main Script

In [1]:
import json
import re
import random

import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.pred_wrapper import PredictorWrapper
from allennlp_models.pretrained import load_predictor
import warnings
warnings.filterwarnings('ignore')



In [2]:
%run utils.ipynb
%run generation_voice.ipynb
%run generation_long_distance.ipynb
%run generation_loc_vs_dir.ipynb
%run generation_dativ_alternation.ipynb
%run generation_other_alternations.ipynb

In [3]:
'''
# Initialize random seed

import torch
seed = 423
random.seed(seed)
torch.manual_seed(seed)
'''

'\n# Initialize random seed\n\nimport torch\nseed = 423\nrandom.seed(seed)\ntorch.manual_seed(seed)\n'

In [4]:
bert_predictor = load_predictor('structured-prediction-srl-bert') 

lstm_predictor = load_predictor('structured-prediction-srl') 


2022-04-01 02:49:57,126 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-04-01 02:49:57,219 - INFO - allennlp.common.params - id = vgqa-vilbert
2022-04-01 02:49:57,220 - INFO - allennlp.common.params - registered_model_name = vqa_vilbert_from_huggingface
2022-04-01 02:49:57,220 - INFO - allennlp.common.params - model_class = None
2022-04-01 02:49:57,221 - INFO - allennlp.common.params - registered_predictor_name = vgqa_vilbert
2022-04-01 02:49:57,222 - INFO - allennlp.common.params - display_name = ViLBERT - Visual Genome Question Answering
2022-04-01 02:49:57,224 - INFO - allennlp.common.params - task_id = vgqa
2022-04-01 02:49:57,233 - INFO - allennlp.common.params - model_usage.archive_file = vilbert-vgqa-pretrained.2021-05-10.tar.gz
2022-04-01 02:49:57,235 - INFO - allennlp.common.params - model_usage.training_config = vision/vilbert_vgqa_pretrained.jsonnet
2022-04-01 02:49:57,236 - INFO - allennlp.common.params - model_usage.install_instructions = pip insta

2022-04-01 02:49:57,677 - INFO - allennlp.common.params - model_details.paper.citation = 
@article{Shi2019SimpleBM,
title={Simple BERT Models for Relation Extraction and Semantic Role Labeling},
author={Peng Shi and Jimmy Lin},
journal={ArXiv},
year={2019},
volume={abs/1904.05255}}

2022-04-01 02:49:57,684 - INFO - allennlp.common.params - model_details.paper.title = Simple BERT Models for Relation Extraction and Semantic Role Labeling
2022-04-01 02:49:57,685 - INFO - allennlp.common.params - model_details.paper.url = https://api.semanticscholar.org/CorpusID:131773936
2022-04-01 02:49:57,688 - INFO - allennlp.common.params - model_details.license = None
2022-04-01 02:49:57,690 - INFO - allennlp.common.params - model_details.contact = allennlp-contact@allenai.org
2022-04-01 02:49:57,698 - INFO - allennlp.common.params - intended_use.primary_uses = None
2022-04-01 02:49:57,704 - INFO - allennlp.common.params - intended_use.primary_users = None
2022-04-01 02:49:57,714 - INFO - allennlp.co

2022-04-01 02:49:58,331 - INFO - allennlp.common.params - model_class = None
2022-04-01 02:49:58,332 - INFO - allennlp.common.params - registered_predictor_name = None
2022-04-01 02:49:58,333 - INFO - allennlp.common.params - display_name = Deep Biaffine Attention for Neural Dependency Parsing
2022-04-01 02:49:58,333 - INFO - allennlp.common.params - task_id = dependency-parsing
2022-04-01 02:49:58,336 - INFO - allennlp.common.params - model_usage.archive_file = biaffine-dependency-parser-ptb-2020.04.06.tar.gz
2022-04-01 02:49:58,337 - INFO - allennlp.common.params - model_usage.training_config = structured_prediction/dependency_parser.jsonnet
2022-04-01 02:49:58,339 - INFO - allennlp.common.params - model_usage.install_instructions = pip install allennlp==2.1.0 allennlp-models==2.1.0
2022-04-01 02:49:58,340 - INFO - allennlp.common.params - model_usage.overrides = None
2022-04-01 02:49:58,345 - INFO - allennlp.common.params - model_details.description = This dependency parser follows 

2022-04-01 02:49:58,647 - INFO - allennlp.common.params - model_details.paper.title = RoBERTa: A Robustly Optimized BERT Pretraining Approach (Liu et al)
2022-04-01 02:49:58,649 - INFO - allennlp.common.params - model_details.paper.url = https://api.semanticscholar.org/CorpusID:198953378
2022-04-01 02:49:58,652 - INFO - allennlp.common.params - model_details.license = None
2022-04-01 02:49:58,653 - INFO - allennlp.common.params - model_details.contact = allennlp-contact@allenai.org
2022-04-01 02:49:58,656 - INFO - allennlp.common.params - intended_use.primary_uses = None
2022-04-01 02:49:58,661 - INFO - allennlp.common.params - intended_use.primary_users = None
2022-04-01 02:49:58,664 - INFO - allennlp.common.params - intended_use.out_of_scope_use_cases = None
2022-04-01 02:49:58,670 - INFO - allennlp.common.params - factors.relevant_factors = None
2022-04-01 02:49:58,681 - INFO - allennlp.common.params - factors.evaluation_factors = None
2022-04-01 02:49:58,684 - INFO - allennlp.commo

2022-04-01 02:49:59,119 - INFO - allennlp.common.params - id = roberta-sst
2022-04-01 02:49:59,120 - INFO - allennlp.common.params - registered_model_name = None
2022-04-01 02:49:59,121 - INFO - allennlp.common.params - model_class = None
2022-04-01 02:49:59,121 - INFO - allennlp.common.params - registered_predictor_name = None
2022-04-01 02:49:59,122 - INFO - allennlp.common.params - display_name = RoBERTa large
2022-04-01 02:49:59,125 - INFO - allennlp.common.params - task_id = sentiment-analysis
2022-04-01 02:49:59,132 - INFO - allennlp.common.params - model_usage.archive_file = stanford-sentiment-treebank-roberta.2021-03-11.tar.gz
2022-04-01 02:49:59,133 - INFO - allennlp.common.params - model_usage.training_config = classification/stanford_sentiment_treebank_roberta.jsonnet
2022-04-01 02:49:59,135 - INFO - allennlp.common.params - model_usage.install_instructions = pip install allennlp==2.4.0 allennlp-models==2.4.0
2022-04-01 02:49:59,136 - INFO - allennlp.common.params - model_us

...

2022-04-01 02:50:45,021 - INFO - allennlp.common.params - registered_predictor_name = None
2022-04-01 02:50:45,022 - INFO - allennlp.common.params - display_name = RoBERTa Common Sense QA
2022-04-01 02:50:45,023 - INFO - allennlp.common.params - task_id = mc
2022-04-01 02:50:45,025 - INFO - allennlp.common.params - model_usage.archive_file = commonsenseqa.2020-07-08.tar.gz
2022-04-01 02:50:45,026 - INFO - allennlp.common.params - model_usage.training_config = mc/commonsenseqa.jsonnet
2022-04-01 02:50:45,028 - INFO - allennlp.common.params - model_usage.install_instructions = pip install allennlp==1.0.0 allennlp-models==1.0.0
2022-04-01 02:50:45,029 - INFO - allennlp.common.params - model_usage.overrides = None
2022-04-01 02:50:45,034 - INFO - allennlp.common.params - model_details.description = This is a multiple choice model patterned after the BERT architecture. It calculates a score for each sequence on top of the CLS token, and then chooses the alternative with the highest score.
2

2022-04-01 02:50:45,494 - INFO - allennlp.common.params - metrics.model_performance_measures = Accuracy
2022-04-01 02:50:45,504 - INFO - allennlp.common.params - metrics.decision_thresholds = None
2022-04-01 02:50:45,507 - INFO - allennlp.common.params - metrics.variation_approaches = None
2022-04-01 02:50:45,509 - INFO - allennlp.common.params - evaluation_data.dataset.name = Multi-genre Natural Language Inference (MultiNLI) dev set
2022-04-01 02:50:45,511 - INFO - allennlp.common.params - evaluation_data.dataset.processed_url = https://allennlp.s3.amazonaws.com/datasets/multinli/multinli_1.0_dev_mismatched.jsonl
2022-04-01 02:50:45,516 - INFO - allennlp.common.params - evaluation_data.dataset.url = https://cims.nyu.edu/~sbowman/multinli/
2022-04-01 02:50:45,520 - INFO - allennlp.common.params - evaluation_data.motivation = None
2022-04-01 02:50:45,523 - INFO - allennlp.common.params - evaluation_data.preprocessing = None
2022-04-01 02:50:45,524 - INFO - allennlp.common.params - train

2022-04-01 02:50:45,893 - INFO - allennlp.common.params - model_caveats_and_recommendations.caveats_and_recommendations = None
2022-04-01 02:50:46,046 - INFO - allennlp.common.params - id = tagging-fine-grained-crf-tagger
2022-04-01 02:50:46,047 - INFO - allennlp.common.params - registered_model_name = crf_tagger
2022-04-01 02:50:46,054 - INFO - allennlp.common.params - model_class = None
2022-04-01 02:50:46,058 - INFO - allennlp.common.params - registered_predictor_name = None
2022-04-01 02:50:46,060 - INFO - allennlp.common.params - display_name = Fine Grained Named Entity Recognition
2022-04-01 02:50:46,064 - INFO - allennlp.common.params - task_id = ner
2022-04-01 02:50:46,069 - INFO - allennlp.common.params - model_usage.archive_file = fine-grained-ner.2021-02-11.tar.gz
2022-04-01 02:50:46,070 - INFO - allennlp.common.params - model_usage.training_config = tagging/fine-grained-ner.jsonnet
2022-04-01 02:50:46,075 - INFO - allennlp.common.params - model_usage.install_instructions = 

2022-04-01 02:50:46,375 - INFO - allennlp.common.params - metrics.model_performance_measures = Accuracy
2022-04-01 02:50:46,380 - INFO - allennlp.common.params - metrics.decision_thresholds = None
2022-04-01 02:50:46,381 - INFO - allennlp.common.params - metrics.variation_approaches = None
2022-04-01 02:50:46,391 - INFO - allennlp.common.params - evaluation_data.dataset.name = Stanford Sentiment Treebank
2022-04-01 02:50:46,396 - INFO - allennlp.common.params - evaluation_data.dataset.processed_url = https://allennlp.s3.amazonaws.com/datasets/sst/test.txt
2022-04-01 02:50:46,397 - INFO - allennlp.common.params - evaluation_data.dataset.url = https://nlp.stanford.edu/sentiment/treebank.html
2022-04-01 02:50:46,401 - INFO - allennlp.common.params - evaluation_data.motivation = None
2022-04-01 02:50:46,403 - INFO - allennlp.common.params - evaluation_data.preprocessing = None
2022-04-01 02:50:46,406 - INFO - allennlp.common.params - training_data.dataset.name = Stanford Sentiment Treebank

2022-04-01 02:50:48,642 - INFO - allennlp.nn.initializers -    binary_feature_embedding.weight
2022-04-01 02:50:48,643 - INFO - allennlp.nn.initializers -    encoder._module.layer_0.cell.input_linearity.bias
2022-04-01 02:50:48,645 - INFO - allennlp.nn.initializers -    encoder._module.layer_0.cell.input_linearity.weight
2022-04-01 02:50:48,647 - INFO - allennlp.nn.initializers -    encoder._module.layer_0.cell.state_linearity.bias
2022-04-01 02:50:48,656 - INFO - allennlp.nn.initializers -    encoder._module.layer_0.cell.state_linearity.weight
2022-04-01 02:50:48,659 - INFO - allennlp.nn.initializers -    encoder._module.layer_1.cell.input_linearity.bias
2022-04-01 02:50:48,660 - INFO - allennlp.nn.initializers -    encoder._module.layer_1.cell.input_linearity.weight
2022-04-01 02:50:48,663 - INFO - allennlp.nn.initializers -    encoder._module.layer_1.cell.state_linearity.bias
2022-04-01 02:50:48,665 - INFO - allennlp.nn.initializers -    encoder._module.layer_1.cell.state_linearity.

## Create and Run Tests

In [8]:
%run utils.ipynb


#for model in srl_predictor_dict.keys():
#print('')

bert_dict = create_result_dict()
lstm_dict = create_result_dict()

predict_and_conf_bert = wrap_predict(predict_srl, bert_predictor) 
predict_and_conf_lstm = wrap_predict(predict_srl, lstm_predictor) 


editor = Editor()
nr_of_samples = 100

# loop through predictor


# generate, execute and store results of tests


# test set 1
print('\n\n\n ## Phenomenon 1 - Voice \n')
tests = generate_voice(editor, nr_of_samples)
print('\n -- BERT -- \n')
bert_dict = run_tests(bert_dict, tests, 'MFT', predict_and_conf_bert)
print('\n -- LSTM -- \n')
lstm_dict = run_tests(lstm_dict, tests, 'MFT', predict_and_conf_lstm)

# test set 2
print('\n\n\n ## Phenomenon 2 - Long Distance Dependencies \n')
tests = generate_long_distance_dependencies(editor, nr_of_samples)
print('\n -- BERT -- \n')
bert_dict = run_tests(bert_dict, tests, 'MFT', predict_and_conf_bert)
print('\n -- LSTM -- \n')
lstm_dict = run_tests(lstm_dict, tests, 'MFT', predict_and_conf_lstm)


# test set 3
print('\n\n\n ## Phenomenon 3 - Location vs Direction Confusion \n')
tests = generate_loc_vs_dir(editor, nr_of_samples)
print('\n -- BERT -- \n')
bert_dict = run_tests(bert_dict, tests, 'MFT', predict_and_conf_bert)
print('\n -- LSTM -- \n')
lstm_dict = run_tests(lstm_dict, tests, 'MFT', predict_and_conf_lstm)



print('\n\n\n ## Phenomenon 4 - Dative Alternation \n')
# test set 4
tests = generate_dative_alternation(editor, nr_of_samples)
print('\n -- BERT -- \n')
bert_dict = run_tests(bert_dict, tests, 'MFT', predict_and_conf_bert)
print('\n -- LSTM -- \n')
lstm_dict = run_tests(lstm_dict, tests, 'MFT', predict_and_conf_lstm)


print('\n\n\n ## Phenomenon 5 - Other Alternations (Negation) - Robustness Test \n')
# test set 5
tests = generate_other_alternations(editor, bert_dict, max_nr_of_samples=100)
tests.run(predict_and_conf_bert)
tests.fail_idxs = evaluate_neg(tests)
bert_dict = append_result_dict_by_test(bert_dict, tests, 'INV')
tests.run(predict_and_conf_lstm, overwrite=True)
tests.fail_idxs = evaluate_neg(tests)
lstm_dict = append_result_dict_by_test(lstm_dict, tests, 'INV')



result_dict = {
    'BERT': bert_dict,
    'LSTM': lstm_dict
}

write_out_result_dict(result_dict, path='data/checklist_results.json')






 ## Phenomenon 1 - Voice 


 -- BERT -- 

Predicting 100 examples
Test cases:      100
Fails (rate):    0 (0.0%)

 -- LSTM -- 

Predicting 100 examples
Test cases:      100
Fails (rate):    16 (16.0%)

Example fails:
[ARG0: The tea] [ARGM-MOD: will] be [V: sloshed] [ARGM-MNR: by Ilse] .
----
[ARG2: The drink] was [V: sloshed] [ARG0: by Tash] .
----
[ARG2: The soda] [ARGM-MOD: will] be [V: sloshed] [ARG0: by Peter] .
----



 ## Phenomenon 2 - Long Distance Dependencies 


 -- BERT -- 

Predicting 100 examples
Test cases:      100
Fails (rate):    7 (7.0%)

Example fails:
Frank and Betty , not having seen each other in a long while , are meeting up again after five years to talk about Betty - the third person of their study group from the their shared time in Santa Ana - and to [V: miss] [ARG1: the good old times] .
----
Frank and Nancy , studying together for a long , long time , are meeting up again after five years to talk about Nancy - the third person of their study group from t

In [9]:
bert_dict

{'capabilities': {'capability1': {'capability_name': 'capability1_capability1_voice',
   'tests': {'test1_Passive_Arg1': {'data': ['The water will have been poured by Tash.',
      'The soda will have been sloshed by Tash.',
      'The coffee is spilled by Ilse.',
      'The liquid will be spilled by someone.',
      'The tea will have been spilled by Peter.',
      'The tea is spilled by Tash.',
      'The tea was poured by me.',
      'The water will be poured by Peter.',
      'The tea will be spilled by Ilse.',
      'The paint will be spilled by someone.',
      'The water will have been spilled by Ilse.',
      'The water will be spilled by Ilse.',
      'The tea is sloshed by someone.',
      'The drink was sloshed by Tash.',
      'The drink is sloshed by me.',
      'The drink was poured by me.',
      'The tea had been sloshed by Ilse.',
      'The coke is sloshed by Peter.',
      'The paint will be poured by Peter.',
      'The water had been poured by someone.',
      'The

In [10]:
lstm_dict

{'capabilities': {'capability1': {'capability_name': 'capability1_capability1_voice',
   'tests': {'test1_Passive_Arg1': {'data': ['The water will have been poured by Tash.',
      'The soda will have been sloshed by Tash.',
      'The coffee is spilled by Ilse.',
      'The liquid will be spilled by someone.',
      'The tea will have been spilled by Peter.',
      'The tea is spilled by Tash.',
      'The tea was poured by me.',
      'The water will be poured by Peter.',
      'The tea will be spilled by Ilse.',
      'The paint will be spilled by someone.',
      'The water will have been spilled by Ilse.',
      'The water will be spilled by Ilse.',
      'The tea is sloshed by someone.',
      'The drink was sloshed by Tash.',
      'The drink is sloshed by me.',
      'The drink was poured by me.',
      'The tea had been sloshed by Ilse.',
      'The coke is sloshed by Peter.',
      'The paint will be poured by Peter.',
      'The water had been poured by someone.',
      'The